# Exploratory Data Analysis

## Import packages

In [445]:
import datetime as dt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from statsmodels.tools import categorical

## Functions

This section brings together all our user-defined functions we'll use throughout this document. These functions address three main areas:
- Data Completeness and Missing Values: These help identify and handle missing data issues that might affect our analysis.
- Data Formatting and Type Consistency: These ensure that our data is formatted consistently and uses the correct data types for accurate analysis.
- Visualization Support: Inspired by the BCQ starter kit, these functions help create clear and informative visualizations.

In [446]:
#### Helper functions
# get name of df
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [447]:
#### Functions 1. Data quality: A. Completeness and Missing values
## Functions
# replace unknown with np.nan
def replace_unknown_object_cols(df: pd.DataFrame, na_values:list = ["unknown"]) -> pd.DataFrame:
    """
    Replaces values "unknown" (or supplied values) with np.NaN in object and string-type columns of a DataFrame.
    Args:
        df (pandas.DataFrame): The DataFrame to modify.
    Returns:
        pd.DataFrame: The DataFrame with the replaced values.
    """
    object_cols = df.select_dtypes(include=[object,"string"]).columns
    default_value = ["unknown"]
    if na_values != default_value:
        na_values = na_values + default_value
    na_values_lower = [x.lower() for x in na_values]
    pattern = "|".join(na_values_lower)
    for col in object_cols:
        df[col] = df[col].str.strip().str.lower()
        df[col] = df[col].replace(pattern, np.nan, regex=True)
    return df

# assess prevalence of missingness
def prevalence_missingness(df: pd.DataFrame) -> pd.Series:
    """
    Calculate the percentage of missing values per column in a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    pd.Series: A Series with column names as the index and the percentage of missing values as values.
    """
    missing_percentage = round(df.isnull().sum() / len(df), 4) * 100
    missing_percentage = missing_percentage.reset_index().copy()
    missing_percentage = missing_percentage.rename(columns={"index":"col_name",0:"prevalence_na"})
    return missing_percentage

In [448]:
#### Functions 2. Data quality: B. Consistency in data formatting
## Functions

# convert to bool those columns whose values are [0,1] or [f,t]
def convert_bool_cols(df:pd.DataFrame) -> pd.DataFrame:
    """Converts columns in a DataFrame to bool if their unique values are [0, 1, np.nan].
    Args:
        df: A pandas DataFrame.
    Returns:
        The DataFrame with qualifying columns converted to bool.
    """
    # for [f,t]
    object_cols = df.select_dtypes(include=[object,"string"]).columns
    for col in df[object_cols].columns:
        df[col] = df.loc[:,col].str.lower().copy()
        if set(df[col].unique()) <= {"f","t", np.nan}:
            df[col] = df.loc[:,col].map({"f": 0, "t": 1}).copy()
    # for [0,1]
    for col in df.columns:
        if set(df[col].unique()) <= {0, 1, np.nan}:
            df[col] = df[col].astype(bool).copy()
    return df

# convert string columns matching 'date' to pd.datetime
def convert_obj_to_date(df:pd.DataFrame) -> pd.DataFrame:
    """Converts object columns containing dates to datetime format in a pandas DataFrame.
    Args:
    df (pandas.DataFrame): The DataFrame to convert.
    Returns:
    pandas.DataFrame: The DataFrame with object columns converted to datetime.
    """
    cols_date = [col for col in df.columns if 'date' in col]
    for col in df[cols_date].select_dtypes(include=[object,"string"]):
        try:
            df[col] = pd.to_datetime(df[col]).copy()
        except:
            pass
    return df

# convert string columns to category
def convert_to_category(df:pd.DataFrame, nunique_cutoff:int = 12) -> pd.DataFrame:
    """Converts string columns to pandas category if the column has less than 10 unique values.
    Args:
    df (pandas.DataFrame): The DataFrame to convert.
    Returns:
    pandas.DataFrame: The DataFrame with string columns converted to category if applicable.
    """
    for col in df.select_dtypes(include=[object,"string"]):
        if df[col].nunique() <= nunique_cutoff:
            df[col] = df[col].astype("category")
    return df



In [449]:
#### Function 3. Feature engineer

# function that avoids repeated column names when joining aggregated dataframes
def append_suffix(df:pd.DataFrame, columns:list, suffix:str) -> pd.DataFrame:
    """
    Appends a suffix to the names of selected columns in a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame to modify.
        columns (list): A list of column names to modify.
        suffix (str): The suffix to append.

    Returns:
        pandas.DataFrame: A new DataFrame with modified column names.
    """

    new_cols = [col + suffix for col in columns if col in df.columns]
    return df.rename(columns=dict(zip(columns, new_cols)))

# Function to calculate the correlation coefficients for the lower off-diagonal elements
def correlation_matrix(df:pd.DataFrame, numeric_cols:list) -> pd.DataFrame:
    """
    Calculates the correlation matrix for a subset of numeric columns in a DataFrame.
    Args:
        df (pandas.DataFrame): The DataFrame to analyze.
        cols (list): A list of column names for which to calculate correlations.
    Returns:
        pandas.DataFrame: A DataFrame containing the correlation coefficients.
    """
    # Create a correlation matrix for the numeric columns
    corr_matrix = df[numeric_cols].corr()
    # # Extract column names for the upper triangle of the correlation matrix (excluding redundant correlations)
    upper_tri_cols = [(col1, col2) for col1 in corr_matrix.columns for col2 in corr_matrix.columns if col1 > col2]
    # Create a DataFrame to store correlation coefficients
    corr_df = pd.DataFrame(columns=["name_col1", "name_col2", "correlation"])
    # Fill the DataFrame with correlation coefficients and column names
    for i, (col1, col2) in enumerate(upper_tri_cols):
        corr_df.loc[i] = [col1, col2, corr_matrix.loc[col1, col2]]
    corr_df = corr_df.loc[corr_df["correlation"] < 1].copy()
    return corr_df

# Function that reduces a set of numerical columns to its most important PC
def extract_principal_components(df, cols):
    """
    Extracts principal components from a DataFrame using selected columns.
    Args:
        df (pandas.DataFrame): The DataFrame containing the data.
        cols (list): A list of column names to use for PCA.
    Returns:
        pandas.DataFrame: A DataFrame containing the principal components.
    """
    
    # Scale the data
    scaled_data = scale(df[cols])
    # Create a PCA object
    pca = PCA()
    # Fit the PCA model to the scaled data
    pca.fit(scaled_data)
    # Transform the scaled data to principal components
    principal_components = pca.transform(scaled_data)
    # Create a DataFrame to store the principal components
    pc_df = pd.DataFrame(principal_components, columns=[f"pc_{i+1}" for i in range(len(cols))])
    # choose number of components that explain 95% of variance 
    pc_explained_var = np.cumsum(pca.explained_variance_ratio_)
    n_components_95 = np.argmax(pc_explained_var >= 0.95) + 1
    # Reduce the number of components to those that explain 95% of the variance
    pc_df = pc_df.iloc[:, :n_components_95]
    return pc_df


---

## 1. Load
We load the data with `convert_types()` to ensure precise data type definitions.

In [450]:
client_df = pd.read_csv('data/clean_data_after_eda.csv').convert_dtypes()
price_df = pd.read_csv('data/price_data.csv').convert_dtypes()

We use `info()` and `head()` to get a basic understanding of our two main datasets: `client_df` and `price_df`.

**Main Takeaways:**

* **Size:** `client_df` has 43 columns and 14606 rows, while `price_df` has 8 columns and 193002 rows.

* **Content:** `client_df` holds client characteristics, while `price_df` is a time series of prices.

* **Connection:** Both data frames share a primary key named `id`.

* **Missing Values:** While initial inspection suggests no missing values (non-null count equals entries), further analysis might reveal imputed values.

* **Date Format:** Date columns in both data frames are currently strings and need conversion to datetime format.

* **Data Types:** `price_df` contains mostly numerical data (besides date data and `id`), while `client_df` has a combination of numerical and categorical data.

* **Categorical Variables in `client_df`:**
    * **Dichotomous:** Variables like `churn` and `has_gas` are currently stored as integer and string, respectively. We'll convert them to booleans to improve readability and consistency.
    * **Nominal:** Columns like `channel_sales` and `origin_up` have nominal categorical data. We'll convert them to the `categorical` type for better handling and readability.

In [451]:
client_df.info(verbose=True, max_cols=30)
client_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 44 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   id                              string 
 1   channel_sales                   string 
 2   cons_12m                        Int64  
 3   cons_gas_12m                    Int64  
 4   cons_last_month                 Int64  
 5   date_activ                      string 
 6   date_end                        string 
 7   date_modif_prod                 string 
 8   date_renewal                    string 
 9   forecast_cons_12m               Float64
 10  forecast_cons_year              Int64  
 11  forecast_discount_energy        Int64  
 12  forecast_meter_rent_12m         Float64
 13  forecast_price_energy_off_peak  Float64
 14  forecast_price_energy_peak      Float64
 15  forecast_price_pow_off_peak     Float64
 16  has_gas                         string 
 17  imp_cons                       

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.000131,0.000041,0.000908,2.086294,99.530517,44.235794,2.086425,99.530558,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,0.001218,0.0,0.009482,0.0,0.0,0.009485,0.001218,0.0,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,0.0,0.0,0.0,0.0,0.0,0.000004,0.0,0.0,0


In [452]:
price_df.info()
price_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193002 entries, 0 to 193001
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  193002 non-null  string 
 1   price_date          193002 non-null  string 
 2   price_off_peak_var  193002 non-null  Float64
 3   price_peak_var      193002 non-null  Float64
 4   price_mid_peak_var  193002 non-null  Float64
 5   price_off_peak_fix  193002 non-null  Float64
 6   price_peak_fix      193002 non-null  Float64
 7   price_mid_peak_fix  193002 non-null  Float64
dtypes: Float64(6), string(2)
memory usage: 12.9 MB


,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0


---

## 2. Data cleaning and preparation

### Data quality: Completeness and Missing values

This section focuses on finding and handling missing values in the categorical columns.

We use the `channel_sales` column as our starting point. The output of `head()` below shows that it has a value of `"MISSING"` to indicate missing data (NA). We'll check if other categorical columns use similar values (e.g., "NA", "Unknown") to represent NA.

Since categorical data in these columns is stored as hashed text strings (refer to the data's documentation), they usually have a fixed length. We leverage this characteristic to identify missing values. We compare the length of each string to the maximum length within the column. Entries with a shorter length might indicate missing values that have been hashed with a special value. In contrast, we consider that if all values share the same length, NA are absent.

All values identified correspond to 'MISSING'. Therefore, we replace these values with `np.nan` for consistency.

In [453]:
client_df[client_df.select_dtypes(include=['string']).columns].head(3)

,id,channel_sales,date_activ,date_end,date_modif_prod,date_renewal,has_gas,origin_up
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,2013-06-15,2016-06-15,2015-11-01,2015-06-23,t,lxidpiddsbxsbosboudacockeimpuepw
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,2009-08-21,2016-08-30,2009-08-21,2015-08-31,f,kamkkxfxxuwbdslkwifmmcsiusiuosws
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,2010-04-16,2016-04-16,2010-04-16,2015-04-17,f,kamkkxfxxuwbdslkwifmmcsiusiuosws


**First**, we iterate through all string columns in both dataframes to identify inconsistencies in string lengths and discover which values represent missing data (NA) in these columns.

Our assessment reveals that only `client_df` has missing values, while `price_df` is complete. In both the `channel_sales` and `origin_up` columns within `client_df`, the value `"MISSING"` is used to indicate missing data.


In [454]:
cols_string_missing_values = []
for df in (client_df, price_df):
    cols_string_inconsistent = []
    for col in df.select_dtypes(include=['string']).columns:
        if max(df[col].str.len()) != min(df[col].str.len()):
            cols_string_inconsistent.append(col)
        else:
            continue
    if cols_string_inconsistent:
        for col in cols_string_inconsistent:
            print(f"{get_df_name(df)}:{col} has inconsistent string length: max {max(df[col].str.len())}, min {min(df[col].str.len())}")
    else:
        print(f"{get_df_name(df)} has no inconsistent columns")

    for col in cols_string_inconsistent:
        col_min = min(df[col].str.len())
        client_subset_loop = df[df[col].str.len() == col_min]
        cols_string_missing_values.extend(list(client_subset_loop[col].unique()))
        print(f"{get_df_name(df)}:{col}, strings of min length: {list(client_subset_loop[col].unique())}")
print(f"\nUnique values denoting NA in all string columns of all dfs: {list(set(cols_string_missing_values))}")


client_df:channel_sales has inconsistent string length: max 32, min 7
client_df:origin_up has inconsistent string length: max 32, min 7
client_df:channel_sales, strings of min length: ['MISSING']
client_df:origin_up, strings of min length: ['MISSING']
price_df has no inconsistent columns

Unique values denoting NA in all string columns of all dfs: ['MISSING']


**Second**, we address missing values. We use our custom `replace_unknown_object_cols()` function to replace the string 'MISSING' with `np.nan` in the previously identified columns. This step ensures a consistent representation of missing data across the dataset. We deliberately choose not to retain 'MISSING' as a separate category in these columns. Keeping it would effectively create an artificial attribute unrelated to the actual information these columns are meant to convey, potentially introducing bias into our model. We discuss below how we deal with these missing values.

**Third**, we assess missingness prevalence. We use our custom function `prevalence_missingness()` to analyze the percentage of missing values in each column of both dataframes. The analysis reveals a high proportion of missing values (around 25%) in the `channel_sales` column, while the `origin_up` column has very few missing values (less than 1%).

In [455]:
client_df = replace_unknown_object_cols(client_df, cols_string_missing_values).copy()
prevalence_missingness(client_df)

,col_name,prevalence_na
0,id,0.00
1,channel_sales,25.50
2,cons_12m,0.00
3,cons_gas_12m,0.00
4,cons_last_month,0.00
5,date_activ,0.00
6,date_end,0.00
7,date_modif_prod,0.00
8,date_renewal,0.00
9,forecast_cons_12m,0.00


**Fourth**, we address the relative high prevalence of NA in the `channel_sales` column.  We consider two approaches to tackle this issue:

   - **Option 1: Drop the Column:** This approach prioritizes keeping all observations in the dataset (maximizing sample size) even if it means removing the `channel_sales` column entirely.
   - **Option 2: Drop Observations with Missing Values:** This approach prioritizes keeping the `channel_sales` column (preserving the information it provides) even if it means removing some observations from the dataset (reducing sample size).

We choose the second option for the following two reasons:

   - The relatively large initial sample size of the `client_df` dataframe (14,606 rows) allows for some reduction without significantly impacting analysis.
   - The `channel_sales` column offers valuable information: 
      - It contains categorical nominal data, which is uncommon in our datasets.
      - It directly relates to our core analytical focus: defensive marketing and churn prevention.

Therefore, we opt to remove observations with missing values in the `channel_sales` or in `origin_up` columns to retain these crucial data points for our analysis.

In [456]:
# remove NAs
client_df = client_df.dropna().copy()

### Data quality: Formatting and Type Consistency

First, we addresses data type conversion to enhance data handling, consistency, and readability. We proceed as follows:

* Categorical dichotomous data (stored as integers or strings) to Boolean.
* Date data (stored as strings) to datetime.
* Categorical nominal data (stored as strings) to category.

In [457]:
# convert to bool
client_df = convert_bool_cols(client_df).copy()
price_df = convert_bool_cols(price_df).copy()

# convert to pd.datetime
client_df = convert_obj_to_date(client_df).copy()
price_df = convert_obj_to_date(price_df).copy()

# convert to categorical data
client_df = convert_to_category(client_df).copy()

Second, we identify and address data inconsistencies arising in our categorical and date data.

First, we address inconsistencies in the column `has_gas`, which indicates whether a customer is a gas client. We assume that a customer with positive gas consumption in the past 12 months (reflected in the `cons_gas_12m field` > 0) should be marked as a gas client (`has_gas` = `True`). We identify and correct any discrepancies based on this assumption.

In [458]:
# assess inconsistencies in data
# we drop those cases where "has_gas" and "cons_gas_12m" are inconsistent
client_df["gas_inconsistent"] = [
    True if (a > 0 and not b) else False 
    for a, b in zip(client_df["cons_gas_12m"], client_df["has_gas"])
]
size_gas = len(client_df[client_df["gas_inconsistent"]==True])
if size_gas > 0:
    print(f"Size of inconsistencies in has_gas: {size_gas}")

    # we replace those case that are inconsistent with "True",
    # otherwise if leave them as is
    client_df["has_gas"] = np.where(client_df["gas_inconsistent"], True, client_df["has_gas"]).copy()
    # we may also remove these is necessary
    # client_df = client_df.drop(client_df[client_df["gas_inconsistent"]==True].index).copy()
client_df = client_df.drop(columns=["gas_inconsistent"]).copy()

Size of inconsistencies in has_gas: 41


**Second,** we examine our date data for inconsistencies. We evaluate the following three assumptions:

* **Assumption 1:** `date_activ` (date of contract activation) should be earlier than any other date values, such as `date_end` (registered date of contract termination) or `date_renewal` (date of the next contract renewal). If this assumption is violated, we drop these observations.
* **Assumption 2:** `date_end` (registered date of contract termination) should be the latest date per row. In other words, all other dates related to contract status should be earlier. Inconsistencies result in dropping the observations.
* **Assumption 3:** `num_years_antig` should be equal to `date_end` minus `date_activ`. If this assumption is not met, we estimate and replace the inconsistent values in `num_years_antig`.


In [459]:
# dates
cols_client_date = [col for col in client_df.columns if 'date' in col]

# assumption 1: date_active is the lowest value per row
print("assumption 1: date_active is the lowest value per row")
client_df["date_min"] = client_df[cols_client_date].min(axis=1)
client_df["date_activ_inconsistent"] = [
    True if (a > b) else False 
    for a, b in zip(client_df["date_activ"], client_df["date_min"])
]
size_1 = len(client_df[client_df["date_activ_inconsistent"]==True])
if size_1 > 0:
    print(f"Size of inconsistencies across rows in date_activ: {size_1}")
    client_df = client_df.drop(client_df[client_df["date_activ_inconsistent"]==True].index).copy()
    print(f"Inconsistent cases dropped in date_activ\n")
else:
    print(f"There are no inconsistencies across rows in date_activ\n")

assumption 1: date_active is the lowest value per row
Size of inconsistencies across rows in date_activ: 20
Inconsistent cases dropped in date_activ



In [460]:
# assumption 2: date_end is the highest value per row
print("assumption 2: date_end is the highest value per row")
client_df["date_max"] = client_df[cols_client_date].max(axis=1)
client_df["date_end_inconsistent"] = [
    True if (a < b) else False 
    for a, b in zip(client_df["date_end"], client_df["date_max"])
]
size_2 = len(client_df[client_df["date_end_inconsistent"]==True])
if size_2 > 0:
    print(f"Size of inconsistencies across rows in date_end: {size_2}")
    client_df = client_df.drop(client_df[client_df["date_end_inconsistent"]==True].index).copy()
    print(f"Inconsistent cases dropped in date_end\n")
else:
    print(f"There are no inconsistencies across rows in date_end\n")
client_df = client_df.drop(columns=["date_min","date_activ_inconsistent","date_max","date_end_inconsistent"]).copy()

assumption 2: date_end is the highest value per row
There are no inconsistencies across rows in date_end



In [461]:
# assumption 3: num_years_antig is equivalent to date_end - date_activ
print("assumption 3: num_years_antig is equivalent to date_end - date_activ")
client_df["num_years_check"] = client_df["date_end"].dt.year - client_df["date_activ"].dt.year
client_df["num_years_inconsistent"] = [
    True if (a != b) else False 
    for a, b in zip(client_df["num_years_antig"], client_df["num_years_check"])
]
size_3 = len(client_df[client_df["num_years_inconsistent"]==True])
min_3 = client_df["num_years_check"].min()
max_3 = client_df["num_years_check"].max()
if size_3 > 0:
    print(f"Size of inconsistencies across rows in num_years_antig: {size_3}")
    print(f"Min inconsistency per row: {min_3} years")
    print(f"Max inconsistency per row: {max_3} years")
    
    # we replace values with our estimates
    client_df["num_years_antig"] = client_df["num_years_check"].astype("Int64")
    print(f"Inconsistent cases replaced with true estimates in num_years_antig")
else:
    print(f"There are no inconsistencies across rows in num_years_antig")
client_df = client_df.drop(columns=["num_years_check","num_years_inconsistent"]).copy()

assumption 3: num_years_antig is equivalent to date_end - date_activ
Size of inconsistencies across rows in num_years_antig: 4863
Min inconsistency per row: 2 years
Max inconsistency per row: 11 years
Inconsistent cases replaced with true estimates in num_years_antig


### Data quality: Duplicates

Lastly, we assess duplicated entries. We do not find any duplicates.

In [462]:
print("Count of duplicates in client_df:",client_df.duplicated(keep='first').sum())
print("Count of duplicates in price_df:",price_df.duplicated(keep='first').sum())

Count of duplicates in client_df: 0
Count of duplicates in price_df: 0


### The cleaned data

In [463]:
print("Dimensions of client_df:",client_df.shape)
client_df.dtypes.value_counts()

Dimensions of client_df: (10811, 44)


Float64           28
Int64              7
datetime64[ns]     4
bool               2
string[python]     1
category           1
category           1
Name: count, dtype: int64

In [464]:
print("Dimensions of client_df:",price_df.shape)
price_df.dtypes.value_counts()

Dimensions of client_df: (193002, 8)


Float64           6
string[python]    1
datetime64[ns]    1
Name: count, dtype: int64

---

## 3. Feature engineering

As noted earlier, our working hypothesis is that price increases affect customer churn. First, we focus on the development of variables related to prices.

### Prices

First, we develop new variables based on `price_df`. To facilitate readability, we replace the suffixes with appropriate names, according to the data documentation.


In [465]:
# Function to replace suffixes in column names
def replace_suffixes(col_name):
    if '_var' in col_name:
        return col_name.replace('_var', '_energy')
    elif '_fix' in col_name:
        return col_name.replace('_fix', '_power')
    else:
        return col_name

# Applying the function to each column name
price_df.columns = [replace_suffixes(col) for col in price_df.columns]

price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193002 entries, 0 to 193001
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id                     193002 non-null  string        
 1   price_date             193002 non-null  datetime64[ns]
 2   price_off_peak_energy  193002 non-null  Float64       
 3   price_peak_energy      193002 non-null  Float64       
 4   price_mid_peak_energy  193002 non-null  Float64       
 5   price_off_peak_power   193002 non-null  Float64       
 6   price_peak_power       193002 non-null  Float64       
 7   price_mid_peak_power   193002 non-null  Float64       
dtypes: Float64(6), datetime64[ns](1), string(1)
memory usage: 12.9 MB


All dates correspond the 1st of each month. Therefore, each row of data is equivalent to the monthly price.

In [466]:
# descriptive statistics only for datetime columns
print(price_df.describe(include=["datetime"]))
# the unique values in datetime column
print(*price_df["price_date"].unique().tolist(), sep="\n")

                          price_date
count                         193002
mean   2015-06-16 12:50:49.933161216
min              2015-01-01 00:00:00
25%              2015-04-01 00:00:00
50%              2015-07-01 00:00:00
75%              2015-10-01 00:00:00
max              2015-12-01 00:00:00
2015-01-01 00:00:00
2015-02-01 00:00:00
2015-03-01 00:00:00
2015-04-01 00:00:00
2015-05-01 00:00:00
2015-06-01 00:00:00
2015-07-01 00:00:00
2015-08-01 00:00:00
2015-09-01 00:00:00
2015-10-01 00:00:00
2015-11-01 00:00:00
2015-12-01 00:00:00


To make it easier to create new price-based features and group data for calculations, we prepare the data by extracting the year and month.

In [467]:
# Extract year and month as separate columns from the 'price_date' column
price_df["price_year"] = price_df["price_date"].dt.year.astype("category")  # Year as categorical variable
price_df["price_month"] = price_df["price_date"].dt.month.astype("category")  # Month as categorical variable

# Reorder columns to group 'id', 'price_year', 'price_month' at the front while preserving the order of other columns
price_df = price_df[["id","price_year","price_month"] + [c for c in price_df if c not in ["id","price_year","price_month"]]]
# Drop the 'price_date' column as year and month are already extracted
price_df = price_df.drop(columns=["price_date"]).copy()

# Select columns containing numerical data (float64) for processing
numeric_cols = price_df.select_dtypes(include="Float64").columns

#### Difference between prices in December and preceding January

To investigate a colleague's idea that the difference in off-peak electricity prices between December and the previous January might predict customer churn, we calculate both the absolute change and the percentage change in all electricity and power prices over the year 2015.

##### Absolute 12-month growth

In [468]:
# Calculate the absolute difference in prices for a 12 month period: i.e., December (12) and January (1)
price_df_abs_growth_12month = pd.concat([
    # Select the 'id' column from the price_df DataFrame
    price_df["id"],
    # Filter for rows where 'price_month' is in December (12) or January (1)
    # and calculate percentage change for 'numeric_cols' grouped by 'id'
    price_df[price_df["price_month"].isin([1, 12])].groupby(["id"])[numeric_cols].transform("diff").dropna()
    ], axis=1).dropna()
# output
price_df_abs_growth_12month.head(2)

,id,price_off_peak_energy,price_peak_energy,price_mid_peak_energy,price_off_peak_power,price_peak_power,price_mid_peak_power
11,038af19179925da21a25619c5a24b745,-0.005508,0.0,0.0,0.177779,0.0,0.0
23,31f2ce549924679a3cbb2d128ae9ea43,-0.007221,-0.002324,0.00356,0.162916,0.097749,0.065166


##### Relative 12-month growth

In [469]:
# Calculate the percentage change in prices for a 12 month period: i.e., December (12) and January (1)
price_df_pct_growth_12month = pd.concat([
    price_df["id"],
    # Filter for rows where 'price_month' is in December (12) or January (1)
    # and calculate percentage change for 'numeric_cols' grouped by 'id'
    price_df[price_df["price_month"].isin([1, 12])].groupby(["id"])[numeric_cols].transform("pct_change").dropna(),
    ], axis=1).dropna()

# Fill NaN values in 'price_df_pct_growth_12month' with 0; this denotes (0-0)/0
price_df_pct_growth_12month[np.isnan(price_df_pct_growth_12month[numeric_cols])] = 0
# Fill +Inf values in 'price_df_pct_growth_12month' with 1; this denotes (positive-0)/0
price_df_pct_growth_12month = price_df_pct_growth_12month.replace([np.inf], 1)
# Multiply the percentage change by 100 to convert it to a percentage
price_df_pct_growth_12month[numeric_cols] = price_df_pct_growth_12month[numeric_cols] * 100
# output
price_df_pct_growth_12month.head(2)

,id,price_off_peak_energy,price_peak_energy,price_mid_peak_energy,price_off_peak_power,price_peak_power,price_mid_peak_power
11,038af19179925da21a25619c5a24b745,-3.638838,0.0,0.0,0.401606,0.0,0.0
23,31f2ce549924679a3cbb2d128ae9ea43,-5.732044,-2.247691,4.976515,0.401607,0.401607,0.401607


#### Average monthly growth
To further investigate whether price changes are a significant factor in customer churn, we also calculate the average monthly growth rates of customer churn.

##### Average absolute monthly growth

In [470]:
# Calculate the monthly absolute difference in prices
price_df_abs_growth_monthly = pd.concat([
    price_df["id"],
    # Calculate monthly difference for each numeric column by group 'id'
    price_df.groupby(["id"])[numeric_cols].transform("diff").dropna()
    ], axis=1).dropna()
# Calculate average monthly absolute difference by group 'id' and reset index
price_df_abs_growth_monthly_avg = price_df_abs_growth_monthly.groupby(["id"])[numeric_cols].agg("mean").reset_index()
# output
price_df_abs_growth_monthly_avg.head(2)

,id,price_off_peak_energy,price_peak_energy,price_mid_peak_energy,price_off_peak_power,price_peak_power,price_mid_peak_power
0,0002203ffbb812588b632b9e628cc38d,-0.000563,-0.000209,0.000317,0.014811,0.008886,0.005924
1,0004351ebdd665e6ee664792efc4fd13,-0.000373,0.0,0.0,0.016162,0.0,0.0


##### Average relative monthly growth

In [471]:
# Calculate the monthly percentage change in prices
price_df_pct_growth_monthly = pd.concat([
    price_df["id"],
    # Calculate the percentage change for each numeric column by group 'id'
    price_df.groupby(["id"])[numeric_cols].transform("pct_change").dropna()
    ], axis=1).dropna()
# Replace NaN values with 0 for numeric columns
price_df_pct_growth_monthly[np.isnan(price_df_pct_growth_monthly[numeric_cols])] = 0
# Fill +Inf values in 'price_df_pct_growth_12month' with 1; this denotes (positive-0)/0
price_df_pct_growth_monthly = price_df_pct_growth_monthly.replace([np.inf], 1)
# Multiply the percentage change by 100 to convert it to a percentage
price_df_pct_growth_monthly[numeric_cols] = price_df_pct_growth_monthly[numeric_cols] * 100
# Calculate average monthly percentage change by group 'id' and reset index
price_df_pct_growth_monthly_avg = price_df_pct_growth_monthly.groupby(["id"])[numeric_cols].agg("mean").reset_index()
# output
price_df_pct_growth_monthly_avg.head(2)

,id,price_off_peak_energy,price_peak_energy,price_mid_peak_energy,price_off_peak_power,price_peak_power,price_mid_peak_power
0,0002203ffbb812588b632b9e628cc38d,-0.43736,-0.194842,0.451697,0.03651,0.03651,0.03651
1,0004351ebdd665e6ee664792efc4fd13,-0.251347,0.0,0.0,0.03651,0.0,0.0


#### Joining growth data

In [472]:
# Create lists containing dataframes with suffixes added to numeric columns
dfs = (
    append_suffix(price_df_abs_growth_12month, numeric_cols, "_absgrowth_12month"),
    append_suffix(price_df_pct_growth_12month, numeric_cols, "_pctgrowth_12month"),
    append_suffix(price_df_abs_growth_monthly_avg, numeric_cols, "_absgrowth_monthavg"),
    append_suffix(price_df_pct_growth_monthly_avg, numeric_cols, "_pctgrowth_monthavg")
)
# Initial merge of the first two dataframes in the list
data_prices = pd.merge(left=dfs[0], right=dfs[1], on="id")
# Loop through remaining dataframes in the list and perform merge
for df in dfs[2:]:
    data_prices = pd.merge(left=data_prices, right=df, on="id")

# unique user-companies
print("Number of unique user-companies:", data_prices["id"].nunique())
# Count rows with NaN in numeric columns
print("Rows with NaN: ", len(data_prices[np.isnan(data_prices.select_dtypes(include=["Float64"])).any(axis=1)]))
# Count rows with any null value
print("Rows with NA: ", len(data_prices[data_prices.isnull().any(axis=1)]))
# Shape of the final dataframe
data_prices.info()


Number of unique user-companies: 16068
Rows with NaN:  0
Rows with NA:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16068 entries, 0 to 16067
Data columns (total 25 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id                                        16068 non-null  string 
 1   price_off_peak_energy_absgrowth_12month   16068 non-null  Float64
 2   price_peak_energy_absgrowth_12month       16068 non-null  Float64
 3   price_mid_peak_energy_absgrowth_12month   16068 non-null  Float64
 4   price_off_peak_power_absgrowth_12month    16068 non-null  Float64
 5   price_peak_power_absgrowth_12month        16068 non-null  Float64
 6   price_mid_peak_power_absgrowth_12month    16068 non-null  Float64
 7   price_off_peak_energy_pctgrowth_12month   16068 non-null  Float64
 8   price_peak_energy_pctgrowth_12month       16068 non-null  Float64
 9   price_mid_peak_energy_pctgrowth

#### Reduce price columns to optimize model's efficiency

Our data contained 25 price columns, some of which might be redundant (providing the same information). To improve model efficiency, we addressed this by reducing the number of price variables. 

Here's how we did it:

1. **Identified Collinear Variables:** We first identified which price columns were highly correlated (collinear), meaning they provided very similar information.
2. **Extracted Key Components:** We then used a technique called Principal Component Analysis (PCA) to extract the main underlying factors that explained the variation in these collinear variables. 
3. **Reduced Redundancy:** Finally, we kept these key components (which captured most of the information) and removed the original, redundant price columns from the data. 

This process resulted in a significant reduction in the number of price columns, from 25 to 9. This not only improves the efficiency of our model but also ensures we retain the important information for analysis.


**Identify Collinear Variables**

In [473]:
# Calculate the correlation matrix for the numeric columns in 'data_prices'
numeric_cols = list(data_prices.select_dtypes(include=["Float64"]).columns)
corr_data_prices = correlation_matrix(data_prices, numeric_cols)

# Sort the correlation matrix by the 'correlation' column in descending order (highest correlations first)
corr_data_prices = corr_data_prices.sort_values(["correlation"], ascending=False)

# Get descriptive statistics of the correlation matrix to assess its distribution.
corr_data_prices.describe(include=["Float64"])

# Filter the correlation matrix to only include pairs with correlation >= 0.7 (highly correlated)
corr_data_prices_high = corr_data_prices.loc[corr_data_prices["correlation"] >= 0.7]

# Extract the column names from both 'name_col1' and 'name_col2' columns in the filtered data frame
# Use set union to remove duplicates and get a list of all the highly correlated columns
numeric_high_corr_cols = list(set(corr_data_prices_high["name_col1"]).union(corr_data_prices_high["name_col2"]))

# columns of colleague's analytical interest
cols_colleague = ["price_off_peak_power_absgrowth_12month", "price_off_peak_energy_absgrowth_12month"]

# Filter the highly correlated columns to exclude those of colleague's analytical interest
numeric_high_corr_cols_filtered = [col for col in numeric_high_corr_cols if col not in cols_colleague]


**Extract Key Components and Reduce Redundancy**

Descriptive statistics show that the initial max correlation coefficient was reduced to ~0.6 (relative to ~0.9 earlier).

In [474]:
# Extract principal components from the data with high correlation columns
data_prices_pc_high_corr = extract_principal_components(data_prices, numeric_high_corr_cols_filtered)
data_prices_pc_colleague = extract_principal_components(data_prices, cols_colleague)
# Drop high correlation columns from the original data
data_prices_wpc = data_prices.drop(columns=numeric_high_corr_cols_filtered).copy()
# Join the original data with the principal components
data_prices_wpc = data_prices_wpc.join(data_prices_pc_high_corr)

# Rename columns to facilitate readability
data_prices_wpc.columns = [
    f"price_{col}" if col.startswith("pc_") else col
    for col in data_prices_wpc.columns
]

# Drop colleague columns from the original data
data_prices_wpc = data_prices_wpc.drop(columns=cols_colleague).copy()
# Join the original data with the principal components
data_prices_wpc = data_prices_wpc.join(data_prices_pc_colleague)
# Rename columns to facilitate readability
data_prices_wpc.columns = [
    f"colleague_{col}" if col.startswith("pc_") else col
    for col in data_prices_wpc.columns
]

# Calculate correlation matrix for our newly created dataframe
numeric_cols = list(data_prices_wpc.select_dtypes(include=["Float64"]).columns)
corr_data_prices_wpc = correlation_matrix(data_prices_wpc, numeric_cols)
# Sort correlation matrix by correlation in descending order
corr_data_prices_wpc = corr_data_prices_wpc.sort_values(["correlation"], ascending=False)

data_prices_wpc.info()
# Descriptive statistics for numeric columns of the correlation matrix
corr_data_prices_wpc.describe(include=["Float64"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16068 entries, 0 to 16067
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   16068 non-null  string 
 1   price_peak_power_pctgrowth_12month   16068 non-null  Float64
 2   price_peak_power_pctgrowth_monthavg  16068 non-null  Float64
 3   price_pc_1                           16068 non-null  float64
 4   price_pc_2                           16068 non-null  float64
 5   price_pc_3                           16068 non-null  float64
 6   price_pc_4                           16068 non-null  float64
 7   price_pc_5                           16068 non-null  float64
 8   colleague_pc_1                       16068 non-null  float64
 9   colleague_pc_2                       16068 non-null  float64
dtypes: Float64(2), float64(7), string(1)
memory usage: 1.3 MB


,correlation
count,3.600000e+01
mean,-2.675618e-02
std,2.452396e-01
min,-8.705119e-01
25%,-1.305553e-02
50%,3.158047e-16
75%,1.331128e-02
max,5.859146e-01


### Removing unclear price features
The DataFrame `client_df` contains price data with two prefixes: `forecast_` and `var_`. However, the documentation doesn't clarify the reference date for these values. It's unclear whether:

* `forecast_` values are forecasts relative to 2015, or another date like the maximum date in `client_df`.
* `var_` prices refer to the year before maximum date in `client_df`, or to a date range within `price_df`.

To avoid compromising data integrity and consistency with `price_df`, we choose to remove all columns with these prefixes.

In [475]:
cols2drop = [col for col in client_df.columns if col.startswith(("forecast_","var_"))]
client_df = client_df.drop(columns=cols2drop).copy()

### User-company characteristics: Datetime features

Next, we create new features from the `client_df` data. 

Since Random Forests only accept numerical inputs, we focus on transforming the datetime data. To prevent introducing randomness (noise) that might negatively impact the model, we create categorical features based on the datetime information. These new features aim to capture the most relevant aspects of the original datetime data.

Several datetime columns seem redundant. We consider that `date_activ` captures the same information as `num_years_antiquity`, while `churn` (our target variable) already reflects the information in `date_end`. Additionally, we believe that `date_renewal` does not have a meaningful impact on `churn`. Therefore, we remove `date_activ`, `date_end`, and `date_renewal` from the data.

In contrast, we suspect that the date a customer last modified their product (`date_modif_prod`) might influence their likelihood of churning (`churn`). However, our price data only goes back to 2015. To address this shortcoming, we create a new category called `group_modif_prod` to segment customers based on when they modified their product. This category has three values:

* **1:**  Customer modified the product before 2015 (controls for previous behavior).
* **2:**  Customer modified the product in 2015 (controls price effect).
* **3:**  Customer modified the product after 2015 (controls price effect).

By grouping customers this way, we can analyze the relationship between product modification date and potential price impacts on churn.

In [476]:
# Drop unnecessary columns
client_df = client_df.drop(columns=["date_activ","date_end","date_renewal"]).copy()

# Define conditions based on the year in "date_modif_prod" column
condition_list = [
    client_df["date_modif_prod"].dt.year < 2015,  # Before 2015
    client_df["date_modif_prod"].dt.year == 2015,  # In 2015
    client_df["date_modif_prod"].dt.year > 2015   # After 2015
]
# Define corresponding choices (categories) based on conditions
choice_list = [1, 2, 3]
# Create a new column "group_modif_prod"
client_df["group_modif_prod"] = np.select(condition_list, choice_list, default=np.nan)
# Convert "group_modif_prod" column to categorical type for better handling
client_df["group_modif_prod"] = client_df["group_modif_prod"].astype(int).astype("category")

# Drop the original "date_modif_prod" column (no longer needed)
client_df = client_df.drop(columns=["date_modif_prod"]).copy()

# Confirm NAs in the dataframe
print("NAs:",client_df.isna().sum().sum())
client_df.info()

NAs: 0
<class 'pandas.core.frame.DataFrame'>
Index: 10811 entries, 0 to 14604
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    10811 non-null  string  
 1   channel_sales         10811 non-null  category
 2   cons_12m              10811 non-null  Int64   
 3   cons_gas_12m          10811 non-null  Int64   
 4   cons_last_month       10811 non-null  Int64   
 5   has_gas               10811 non-null  bool    
 6   imp_cons              10811 non-null  Float64 
 7   margin_gross_pow_ele  10811 non-null  Float64 
 8   margin_net_pow_ele    10811 non-null  Float64 
 9   nb_prod_act           10811 non-null  Int64   
 10  net_margin            10811 non-null  Float64 
 11  num_years_antig       10811 non-null  Int64   
 12  origin_up             10811 non-null  category
 13  pow_max               10811 non-null  Float64 
 14  churn                 10811 non-null  bool    
 15  

### Join cleaned dataframe

In [477]:
data_clean = pd.merge(
    client_df,
    data_prices_wpc,
    on="id",
    how="inner"
)
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10791 entries, 0 to 10790
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   id                                   10791 non-null  string  
 1   channel_sales                        10791 non-null  category
 2   cons_12m                             10791 non-null  Int64   
 3   cons_gas_12m                         10791 non-null  Int64   
 4   cons_last_month                      10791 non-null  Int64   
 5   has_gas                              10791 non-null  bool    
 6   imp_cons                             10791 non-null  Float64 
 7   margin_gross_pow_ele                 10791 non-null  Float64 
 8   margin_net_pow_ele                   10791 non-null  Float64 
 9   nb_prod_act                          10791 non-null  Int64   
 10  net_margin                           10791 non-null  Float64 
 11  num_years_antig

## 4. Random Forest ML pre-processing

Preparing the data for the Random Forest algorithm depends on the data type:
- **Continuous data**: Linear algorithms like linear regression assume a linear relationship between target and explanatory variables, often requiring transformations (e.g., natural logarithms) to linearize relationships. BCG recommends applying these transformations before implementing Random Forest. However, [Raschka & Mirjalili](https://www.google.com/search?q=Raschka+%26+Mirjalili++2017+Python+Machine+Learning) note that "[...] an advantage of the decision tree algorithm [which form the basis of Random Forest] is that it does not require any transformation of the features if we are dealing with nonlinear data" (2017: 492). Despite this lack of consensus, we choose to log-transform columns with highly skewed distributions. This allows us to evaluate different models using consistent data preprocessing.
- **Categorical data**
    - *Boolean*: In Python, boolean values (True and False) are directly equivalent to the integers 0 and 1, respectively. Therefore, there is no need to explicitly convert them.
    - *Nominal*: Random forest classifiers implemented with `sklearn.ensemble.RandomForestClassifier` can not handle categorical features directly. To address this shortcoming, we create dummy variables for the categories in each column, where the number of dummy variables for each column is equal to the total number of categories. In turn, we express this as boolean values to improve readability.

#### Continuos data

Descriptive statistics reveal that, in several columns, the mean is substantially higher than the median (50th percentile), suggesting right-skewed distributions with long tails. To address this, we apply a logarithmic transformation to the following columns: `cons_12m`, `cons_gas_12m`, `cons_last_month`, and `imp_cons`.

In [478]:
data_clean.describe(include=["Int64","Float64"])

,cons_12m,cons_gas_12m,cons_last_month,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,price_peak_power_pctgrowth_12month,price_peak_power_pctgrowth_monthavg,price_pc_1,price_pc_2,price_pc_3,price_pc_4,price_pc_5,colleague_pc_1,colleague_pc_2
count,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.000000,10791.000000,10791.000000,10791.000000,10791.000000,10791.000000,10791.000000
mean,175173.452229,28439.159577,17527.753128,155.041536,25.310845,25.307321,1.276619,195.405969,4.941155,18.009185,-0.303615,-0.028146,-0.006298,0.000865,-0.026256,-0.022913,0.009072,0.003772,-0.002968
std,613178.368323,170127.857129,68261.767156,318.317652,21.045083,21.044047,0.680272,340.740333,1.218624,12.839611,8.986255,0.819987,3.298342,1.801486,1.576722,0.208815,0.862353,1.210975,0.672200
min,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,3.3,-100.0,-11.111111,-42.790901,-43.972355,-33.869749,-8.332321,-25.149200,-14.636989,-20.094679
25%,5896.5,0.0,0.0,0.0,14.64,14.64,1.0,52.655,4.0,13.0,0.0,0.0,0.018627,-0.435714,-0.197227,-0.036381,-0.160726,-0.004366,-0.106039
50%,14979.0,0.0,847.0,42.45,22.21,22.21,1.0,116.89,5.0,13.856,0.0,0.0,0.043651,-0.143568,-0.179320,-0.014076,-0.034281,0.132649,0.003884
75%,44206.0,0.0,3632.5,197.495,30.22,30.22,1.0,252.195,6.0,19.05,0.401607,0.03651,0.194180,0.000662,0.372364,0.011215,0.097530,0.272776,0.144767
max,6207104.0,4154590.0,771203.0,9682.89,314.76,314.76,32.0,24570.65,11.0,320.0,100.0,9.999999,43.586055,53.430582,38.308531,3.570205,32.447748,27.236570,21.844747


Log-transformation brings the mean and median closer together. This reduces skewness in the data, which helps satisfy the assumption of linearity between the target and explanatory variables common in methods, such as linear regression.

In [479]:
cols_2log = ["cons_12m","cons_gas_12m","cons_last_month","imp_cons"]
data_clean[cols_2log] = data_clean[cols_2log] + 1
data_clean[cols_2log] = np.log(data_clean[cols_2log])
data_clean.describe(include=["Int64","Float64"])

,cons_12m,cons_gas_12m,cons_last_month,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,price_peak_power_pctgrowth_12month,price_peak_power_pctgrowth_monthavg,price_pc_1,price_pc_2,price_pc_3,price_pc_4,price_pc_5,colleague_pc_1,colleague_pc_2
count,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.0,10791.000000,10791.000000,10791.000000,10791.000000,10791.000000,10791.000000,10791.000000
mean,9.828444,1.716915,5.355709,2.927514,25.310845,25.307321,1.276619,195.405969,4.941155,18.009185,-0.303615,-0.028146,-0.006298,0.000865,-0.026256,-0.022913,0.009072,0.003772,-0.002968
std,1.998298,3.893507,4.075058,2.611064,21.045083,21.044047,0.680272,340.740333,1.218624,12.839611,8.986255,0.819987,3.298342,1.801486,1.576722,0.208815,0.862353,1.210975,0.672200
min,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,3.3,-100.0,-11.111111,-42.790901,-43.972355,-33.869749,-8.332321,-25.149200,-14.636989,-20.094679
25%,8.682284,0.0,0.0,0.0,14.64,14.64,1.0,52.655,4.0,13.0,0.0,0.0,0.018627,-0.435714,-0.197227,-0.036381,-0.160726,-0.004366,-0.106039
50%,9.614471,0.0,6.742881,3.771611,22.21,22.21,1.0,116.89,5.0,13.856,0.0,0.0,0.043651,-0.143568,-0.179320,-0.014076,-0.034281,0.132649,0.003884
75%,10.696638,0.0,8.197952,5.290764,30.22,30.22,1.0,252.195,6.0,19.05,0.401607,0.03651,0.194180,0.000662,0.372364,0.011215,0.097530,0.272776,0.144767
max,15.641205,15.239725,13.555708,9.178219,314.76,314.76,32.0,24570.65,11.0,320.0,100.0,9.999999,43.586055,53.430582,38.308531,3.570205,32.447748,27.236570,21.844747


#### Continuos data: Nominal

In [480]:
cols_nominal = data_clean.select_dtypes(include=["category"]).columns.tolist()
for col in cols_nominal:
    col_encoded = pd.get_dummies(data_clean[col], prefix=col)
    data_clean = data_clean.drop(columns=col).copy()
    data_clean = pd.concat([data_clean, col_encoded], axis = 1)
data_clean.dtypes.value_counts()

boolean           11
Float64           10
float64            7
bool               5
Int64              2
string[python]     1
Name: count, dtype: int64

### Final dataframe

In [444]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10791 entries, 0 to 10790
Data columns (total 36 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              10791 non-null  string 
 1   cons_12m                                        10791 non-null  Float64
 2   cons_gas_12m                                    10791 non-null  Float64
 3   cons_last_month                                 10791 non-null  Float64
 4   has_gas                                         10791 non-null  bool   
 5   imp_cons                                        10791 non-null  Float64
 6   margin_gross_pow_ele                            10791 non-null  Float64
 7   margin_net_pow_ele                              10791 non-null  Float64
 8   nb_prod_act                                     10791 non-null  Int64  
 9   net_margin                             